In [1]:
from lxml import html

from bs4 import BeautifulSoup
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import json
import os
import sys

In [2]:
course_schedule = []

In [3]:
def sync_get_element_by_xpath(xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_element_by_xpath(xpath)

In [4]:
def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses=course_schedule):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results
def search_courses_by_name(name_str, courses=course_schedule):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses=course_schedule):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

def click_book_this_class_now():
    btn = driver.find_element_by_xpath("//a[contains(text(), 'BOOK THIS CLASS NOW')]")
    btn_click()
    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:    
        if cur_date is not course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")

In [5]:
#setup option for chrome profile
chrome_options = Options()
# chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")

#start web driver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.implicitly_wait(60)

wait = WebDriverWait(driver, 30)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [6]:
def sign_in():
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')
    username.send_keys("DN20092360")
    password.send_keys("jj1216")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))
    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()
def log_out():
    click_btn = sync_get_element_by_xpath("//a[contains(text(), 'Logout')]")
    print("click_btn: ", click_btn)
    click_btn.click()
    
def click_book_for_class():
    click_btn = sync_get_element_by_xpath("//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    click_btn.click()

def click_book_this_class_now():
    click_btn = driver.find_element_by_xpath("//a[contains(text(), 'BOOK THIS CLASS NOW')]")
    click_btn.click()

In [7]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [8]:
driver.get(base_url)
# sign_in()
# click_book_for_class()

In [9]:
with open(r'/Users/huohsien/workspace/python/yoga_class_smart_booking/class_schedules/14 September - 20 September 2020.json', "r") as read_file:
    course_schedule = json.load(read_file)

In [10]:
# course_schedule

# c = search_courses_by_date('Mon', course_schedule)
# search_courses_by_teacher('Darr', c)

[{'room': '敦南 4 教室',
  'date': '14 Sep Mon',
  'href': 'https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=733054',
  'name': 'Vinyasa Yoga 流暢瑜珈',
  'teacher': 'Darryl',
  'time': '07:00 - 08:00 PM'},
 {'room': '敦南 4 教室',
  'date': '14 Sep Mon',
  'href': 'https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=733055',
  'name': '動態伸展',
  'teacher': 'Darryl',
  'time': '08:15 - 09:15 PM'},
 {'room': '敦南 9 教室',
  'date': '14 Sep Mon',
  'href': 'https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=733170',
  'name': 'Warm Stretch 溫和伸展',
  'teacher': 'Darryl',
  'time': '05:15 - 06:15 PM'}]

In [11]:
book_datetime = datetime.datetime(2020, 9, 10, 22, 0, 0)
courses_to_be_booked = []

c = search_courses_by_date('Mon', course_schedule)
c = search_courses_by_teacher('Darr', c)
# c = search_courses_by_name('Warm', c)
c = search_courses_by_time('PM', c)

courses_to_be_booked.extend(c)

list_courses_to_be_booked(courses_to_be_booked)

Date: 14 Sep Mon
-----------------------

Name: Warm Stretch 溫和伸展
Teacher: Darryl
Time: 05:15 - 06:15 PM




In [12]:
signin_datetime = book_datetime - datetime.timedelta(seconds=20)

In [13]:
def scheduled_sign_in():
    sign_in()
    click_book_for_class()
    
def reserve_class():
    for course_to_be_booked in courses_to_be_booked:
        book_url = course_to_be_booked['href']
        driver.get(book_url)
        click_book_this_class_now()
        
    time.sleep(30)
    log_out()

In [14]:
# Start the scheduleruler
scheduler = BackgroundScheduler()
scheduler.add_job(scheduled_sign_in, 'date', run_date= signin_datetime, args=[])
scheduler.add_job(reserve_class, 'date', run_date= book_datetime, args=[])
scheduler.start()

In [15]:
# scheduled_sign_in()

In [16]:
# click_book_for_class()

In [17]:
# for course_to_be_booked in courses_to_be_booked:
#     book_url = course_to_be_booked['href']
#     driver.get(book_url)
#     click_book_this_class_now()

In [18]:
# log_out()